In [124]:
import pandas as pd
import numpy as np

In [125]:
search_trend_url ="https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/search_trends_symptoms_dataset/United%20States%20of%20America/2020_US_weekly_symptoms_dataset.csv"
hospitalization_url ="https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/cc_by/aggregated_cc_by.csv"
search_trend_df = pd.read_csv(search_trend_url)
hospitalization_df = pd.read_csv(hospitalization_url)
print(search_trend_df)

    open_covid_region_code country_region_code country_region sub_region_1  \
0                    US-AK                  US  United States       Alaska   
1                    US-AK                  US  United States       Alaska   
2                    US-AK                  US  United States       Alaska   
3                    US-AK                  US  United States       Alaska   
4                    US-AK                  US  United States       Alaska   
..                     ...                 ...            ...          ...   
619                  US-WY                  US  United States      Wyoming   
620                  US-WY                  US  United States      Wyoming   
621                  US-WY                  US  United States      Wyoming   
622                  US-WY                  US  United States      Wyoming   
623                  US-WY                  US  United States      Wyoming   

    sub_region_1_code  sub_region_2  sub_region_2_code        d

In [126]:
search_trend_df = search_trend_df.drop(search_trend_df.columns[[0,1,2,4,5,6]], axis=1)
search_trend_df = search_trend_df.dropna(axis=1, how='all')
search_trend_df = search_trend_df.rename(columns={'sub_region_1':'region_name'})

In [127]:
hospitalization_df = hospitalization_df[['open_covid_region_code','region_name','date','hospitalized_new']]
hospitalization_df = hospitalization_df[hospitalization_df['open_covid_region_code'].str.contains('^US-')]
hospitalization_df = hospitalization_df.drop(hospitalization_df.columns[[0]], axis=1)
regions = hospitalization_df.region_name.unique()

for region in regions:
    region_df = hospitalization_df[hospitalization_df['region_name'] == region]
    if (region_df['hospitalized_new'] == 0).all():
        hospitalization_df = hospitalization_df.drop(hospitalization_df[hospitalization_df.region_name == region].index, axis=0) 

In [128]:
print(search_trend_df)

hospitalization_df['date'] = pd.to_datetime(hospitalization_df['date'], format='%Y-%m-%d')
hospitalization_df = hospitalization_df.groupby(['region_name']).resample('W', on='date',loffset='1d').sum()
hospitalization_df = hospitalization_df.reset_index()

search_trend_df['date'] = pd.to_datetime(search_trend_df['date'], format='%Y-%m-%d')
result_df = search_trend_df.merge(hospitalization_df, on=['region_name', 'date'])

result_df.to_csv('result.csv', index=False)

    region_name        date  symptom:Adrenal crisis  symptom:Ageusia  \
0        Alaska  2020-01-06                   12.69              NaN   
1        Alaska  2020-01-13                    9.56              NaN   
2        Alaska  2020-01-20                     NaN              NaN   
3        Alaska  2020-01-27                   15.31             7.47   
4        Alaska  2020-02-03                    8.81              NaN   
..          ...         ...                     ...              ...   
619     Wyoming  2020-08-31                    5.03             2.79   
620     Wyoming  2020-09-07                    3.36             3.25   
621     Wyoming  2020-09-14                    4.15             4.97   
622     Wyoming  2020-09-21                    5.64             3.67   
623     Wyoming  2020-09-28                    9.79             4.54   

     symptom:Allergic conjunctivitis  symptom:Amblyopia  symptom:Amenorrhea  \
0                                NaN                NaN 